In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import os
import numpy as np
from math import sqrt
from numpy import concatenate
import datetime as dt
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import GRU
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adagrad
from tensorflow.python.keras.optimizers import Adadelta
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.initializers import RandomUniform

from google.colab import files
uploaded = files.upload()

### methods

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[0].shift(-i))
        if i == 0:
            names += [('var1(t)')]
        else:
            names += [('var1(t+%d)' % (i))]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
  
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Saving data_dummies_index to data_dummies_index


In [5]:
#for h in range(1,3,1):
#for i in range(3,4,1):
  
  # Observing script runtime
  startTime = dt.datetime.now()  # observe script running time

  data = pd.read_csv('data_dummies_index', header=0, index_col=0, date_parser=pd.to_datetime)
  data['temp']= data['temp'].shift(-72)    # shift temperature 72h in past to obtain 3 day ahead temperature forecast (temp from 2014-01-04 00:00:00 is switched to 2014-01-01 00:00:00
  data.rename(columns={'temp':'temp_72'}, inplace=True)
  data['hour']= data['hour'] + 1

  # 1.3 Scaling
  scaling = 2 # 0 = No scaling | 1 = MinMax Scaling | 2 = Z-Score

  # Scaling the data between min and max: x_scaled = (x - min(x)) / ( max(x) - min(x)) only with parameters calculated from the train set
  if (scaling == 1):
      min, max = 0, 1
      min_load_train, max_load_train = max(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), min(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00'])
      data['load'] = ((data['load'] - min_load_train) / (max_load_train - min_load_train)) * (max - min) + min    # Inverse scaling: x = (x_scaled - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      min_temp_train, max_temp_train = max(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), min( data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00'])
      data['temp_72'] = ((data['temp_72'] - min_temp_train) / (max_temp_train - min_temp_train)) * (max - min) + min  # Inverse scaling: x = (x_scaled - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

  # Scaling the data as folllows:  x_scaled = (x - mean(x)) / st(x) only with parameters calculated from the train set
  if (scaling == 2):
      mean_load_train, sd_load_train = np.mean(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.sqrt(np.var(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']))
      data['load'] = (data['load'] - mean_load_train) / sd_load_train   # Inverse scaling: x = x_scaled * sd_load_train + mean_load_train
      mean_temp_train, sd_temp_train = np.mean(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.sqrt(np.var(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']))
      data['temp_72'] = (data['temp_72'] - mean_temp_train) / sd_temp_train  # Inverse scaling: x = x_scaled * sd_load_train + mean_load_train
      mean_hour_train, sd_hour_train = np.mean(data['hour']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.sqrt(np.var(data['hour']['2014-01-01 00:00:00':'2015-12-31 23:00:00']))
      data['hour'] = (data['hour'] - mean_hour_train) / sd_hour_train  # Inverse scaling: x = x_scaled * sd_load_train + mean_load_train


  data = data[['load', 'temp_72', 'hour']]     # chose columns to include in forecast (26304, 3) e.g. data = data.drop(['const','weekday','hour'], axis=1)  from 2014-01-01 00:00:00 until 2016-12-31 23:00:00

  # ensure all data is float
  values = data.values.astype('float32')

  # specify the number of lag hours
  train_n = 24*5
  predict_n = 24*3
  n_features = data.shape[1]   # number of input variables

  # data preparation
  # 1: obtain percentage of training set as validation set and then shuffle during training
  # 2: take only every k th sample of training set and obtain a percentage of the randomised training set as validation data 
  preparation = 1 # h 
  p = 0.25 # perceentage of the training data as validation
  k = 1

  if (preparation == 1):
    # split into train, validation and test sets
    train = series_to_supervised(values[ :round(365*2*24*(1-p)), :], train_n, predict_n).values
    val =   series_to_supervised(values[round(365*2*24*(1-p)):365*2*24, :], train_n, predict_n).values
    test =  series_to_supervised(values[365*2*24: , : ], train_n, predict_n).values

  if (preparation == 2):
    train_val = np.array(pd.DataFrame(series_to_supervised(values[:365*2*24, :], train_n, predict_n)).iloc[::k, :])
    np.random.shuffle(train_val)
    train = train_val[:round(len(train_val)*(1-p)), : ]
    val = train_val[round(len(train_val)*(1-p)): , : ]
    test =  np.array(pd.DataFrame(series_to_supervised(values[365*2*24:, :], train_n, predict_n)).iloc[::k, :])


  # split into input and outputs
  input_cols = train_n * n_features    # columns of input tensor
  X_train, y_train = train[:, :input_cols], train[:, -predict_n:]  # (training samples, columns of input tensor) | (training samples, number of periods to predict)
  X_val, y_val = val[:, :input_cols], val[:, -predict_n:]
  X_test, y_test = test[:, :input_cols], test[:, -predict_n:]  # (testing samples, columns of input tensor) | (testing samples samples, number of periods to predict)

  # Selecting only every j-th sample
  j = 1   # j = 24 means you only consider every 24th sample
  X_train, y_train = np.array(pd.DataFrame(X_train).iloc[::j, :]), np.array(pd.DataFrame(y_train).iloc[::j, :])
  X_test, y_test = np.array(pd.DataFrame(X_test).iloc[::j, :]), np.array(pd.DataFrame(y_test).iloc[::j, :])

  # reshape input to be 3D [samples, timesteps, features]
  X_train = X_train.reshape((X_train.shape[0], train_n, n_features))  # (training samples, number of periods to train, number of input variables)
  X_test = X_test.reshape((X_test.shape[0], train_n, n_features)) # # (testing samples, number of periods to train, number of input variables)
  X_val = X_val.reshape((X_val.shape[0], train_n, n_features))  # (training samples, number of periods to train, number of input variables)

  ##### 02 Network selection
  hiddenNeurons = 200

  # architecture
  mv_gru = Sequential()
  mv_gru.add(GRU(units=hiddenNeurons, return_sequences=False, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
  mv_gru.add(Dropout(0.2))

  mv_gru.add(Dense(units=predict_n, activation='linear', kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)))

  # optimizers:
  sgd=SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False, clipnorm=1.0)  # default: SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
  rmsprop=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0) # default:RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
  adagrad=Adagrad(lr=0.01, epsilon=None, decay=0.0)    # default: Adagrad(lr=0.01, epsilon=None, decay=0.0)
  adadelta=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)     # default: Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
  adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999,clipnorm=1.0, epsilon=None, decay=0.0, amsgrad=False)   # default=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  # compilation
  mv_gru.compile(optimizer=adam, loss='mse') # working: 'adam' (relu), 'rmsprop(lr=0.001)'   | not working: sgd (just with tanh)

  # patient early stopping
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

  # fit model
  history = mv_gru.fit(X_train, y_train, batch_size = 200, epochs=1000, verbose=2,
                        validation_data=(X_val, y_val),callbacks=[es], shuffle=True)
  
  print( 'Hidden Neurons: ' + str(hiddenNeurons))
  print( 'Preparation: ' + str(preparation))
  print( 'p: ' + str(p))

  # prediction
  if (scaling == 0):
      yhat = mv_gru.predict(X_test)
      yfit = mv_gru.predict(X_train)  # fitted values
      yval = mv_gru.predict(X_val)  # fitted values

  if (scaling == 1):
      yhat_s = mv_gru.predict(X_test)
      yhat = (yhat_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_test = (y_test - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

      yval_s = mv_gru.predict(X_val)
      yval = (yval_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_val = (y_val - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

      yfit_s = mv_gru.predict(X_train) 
      yfit = (yfit_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_train = (y_train - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

  if (scaling == 2):
      yhat_s = mv_gru.predict(X_test)
      yhat = yhat_s * sd_load_train + mean_load_train
      y_test = y_test * sd_load_train + mean_load_train

      yfit_s = mv_gru.predict(X_train)
      yfit = yfit_s * sd_load_train + mean_load_train
      y_train = y_train  * sd_load_train + mean_load_train

      yval_s = mv_gru.predict(X_val)
      yval = yval_s * sd_load_train + mean_load_train
      y_val = y_val  * sd_load_train + mean_load_train

  # evaluation measures
  rmspe_overall = np.sqrt(mean_squared_error(y_test,yhat)); print('rmspe_overall: ' + str(rmspe_overall))
  rmse_validation_overall = np.sqrt(mean_squared_error(y_val,yval)); print('rmse_validation_all: ' + str(rmse_validation_overall))
  rmse_fitted_overall = np.sqrt(mean_squared_error(y_train,yfit)); print('rmse_fitted_all: ' + str(rmse_fitted_overall))
  mae_overall = mean_absolute_error(y_test,yhat); print('mae_overall: ' + str(mae_overall))
  mape_overall = mean_absolute_percentage_error(y_test,yhat); print('mape_overall: ' + str(mape_overall))

  print('elapsed time: ', dt.datetime.now() - startTime)  # observe script running time

    ## Download model
    #mv_gru.save('mv_gru' + str(np.round(rmspe_overall,2)) + '.h5') 
    #from google.colab import files
    #files.download('mv_gru' + str(np.round(rmspe_overall,2)) + '.h5')

Train on 12949 samples, validate on 4189 samples
Epoch 1/1000
12949/12949 - 67s - loss: 0.9212 - val_loss: 0.3874
Epoch 2/1000
12949/12949 - 65s - loss: 0.2222 - val_loss: 0.0646
Epoch 3/1000
12949/12949 - 65s - loss: 0.1135 - val_loss: 0.0633
Epoch 4/1000
12949/12949 - 65s - loss: 0.0993 - val_loss: 0.0655
Epoch 5/1000
12949/12949 - 65s - loss: 0.0912 - val_loss: 0.0670
Epoch 6/1000
12949/12949 - 66s - loss: 0.0882 - val_loss: 0.0664
Epoch 7/1000
12949/12949 - 65s - loss: 0.0861 - val_loss: 0.0644
Epoch 8/1000
12949/12949 - 65s - loss: 0.0849 - val_loss: 0.0627
Epoch 9/1000
12949/12949 - 65s - loss: 0.0834 - val_loss: 0.0612
Epoch 10/1000
12949/12949 - 66s - loss: 0.0819 - val_loss: 0.0605
Epoch 11/1000
12949/12949 - 66s - loss: 0.0806 - val_loss: 0.0589
Epoch 12/1000
12949/12949 - 66s - loss: 0.0794 - val_loss: 0.0580
Epoch 13/1000
12949/12949 - 65s - loss: 0.0784 - val_loss: 0.0570
Epoch 14/1000
12949/12949 - 65s - loss: 0.0772 - val_loss: 0.0556
Epoch 15/1000
12949/12949 - 66s - lo

In [0]:
# Download y_test and y_hat

pd.DataFrame(yhat).to_csv('y_hat__basic_final') 
from google.colab import files
files.download('y_hat__basic_final')

In [0]:
# Download model
mv_gru.save('mv_gru_basic_final.h5') 
from google.colab import files
files.download("mv_gru_basic_final.h5")

In [0]:
!nvidia-smi

In [0]:
#### Load model
from google.colab import files
uploaded = files.upload()

In [0]:
# Download y_test and y_hat
pd.DataFrame(y_test).to_csv('y_test3__basic_final') 
from google.colab import files
files.download('y_test3__basic_final')

In [0]:
from tensorflow.python.keras.models import load_model
mv_lstm_load = load_model('mv_lstm1272.h5')

In [0]:
mv_lstm_load.summary()

In [0]:
##### 03 Evaluation

# plot history
plt.plot(history.history['loss'][10:], label='train')
plt.plot(history.history['val_loss'][10:], label='test')
plt.legend()

In [0]:
# plot prediction
 for i in range(0, len(y_test), 24*7*4):
     plt.figure(i)
     plt.plot(yhat[i], label='prediction')
     plt.plot(y_test[i], label ='observation')
     plt.title('prediction of week ' + str((round(i/(24*7)+1,0))))
     plt.xlabel('hour');plt.ylabel('MW');plt.legend()
     plt.legend()



In [0]:
# plot "fitted values"
 for i in range(0, len(y_train), 24*7*4):
     plt.figure(i+10)
     plt.plot(yfit[i], label='fitted values')
     plt.plot(y_train[i], label ='observation')
     plt.title('fitted values of week ' + str((round(i/(24*7)+1,0))))
     plt.xlabel('hour');plt.ylabel('MW');plt.legend()
     plt.legend()

In [0]:
# Download model
mv_lstm.save('mv_lstm1400.h5') 
from google.colab import files
files.download("mv_lstm1400.h5")